# Comparison: Julia, PyArray, NLCPy

In [1]:
versioninfo()

Julia Version 1.7.3
Commit 742b9abb4d (2022-05-06 12:58 UTC)
Platform Info:
  OS: Linux (x86_64-redhat-linux)
  CPU: Intel(R) Xeon(R) Gold 6226 CPU @ 2.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, cascadelake)
Environment:
  JULIA_DEPOT_PATH = /home/manabu/.julia-1.7.3
  JULIA_NUM_THREADS = 12


---

In [1]:
using PyCall

In [2]:
ENV["VE_NODE_NUMBER"] = "1"

"1"

In [3]:
np = pyimport("numpy");
vp = pyimport("nlcpy");

## Create array

In [7]:
@. 8*(2^17, 2^27)/1024/1024

(1.0, 1024.0)

In [117]:
# native
@time x0 = rand(Float64, (2^17,2^10));

  0.106476 seconds (2 allocations: 1.000 GiB)


* [JuliaPy/PyCall.jl: Package to call Python functions from the Julia language](https://github.com/JuliaPy/PyCall.jl)
  - "By default, they are passed from Julia to Python without making a copy, but from Python to Julia a copy is made; no-copy conversion of Python to Julia arrays can be achieved with the `PyArray` type below."

In [133]:
# numpy -> native
@time x1 = np.random.random_sample((2^17,2^10));

  2.957480 seconds (59 allocations: 1.000 GiB, 0.15% gc time)


In [129]:
# PyArray (numpy)
@time x2 = @pycall np.random.random_sample((2^17,2^10))::PyArray;

  0.885265 seconds (26 allocations: 1.078 KiB)


* [Profiling · The Julia Language](https://docs.julialang.org/en/v1/manual/profile/)

In [123]:
using Profile
Profile.clear()

In [124]:
# numpy -> native
@profile x1 = np.random.random_sample((2^17,2^10));

In [125]:
Profile.print()

Overhead ╎ [+additional indent] Count File:Line; Function
    ╎2945 @Base/task.jl:429; (::IJulia.var"#15#18")()
    ╎ 2945 ...lia/src/eventloop.jl:8; eventloop(socket::ZMQ.Socket)
    ╎  2945 @Base/essentials.jl:714; invokelatest
    ╎   2945 @Base/essentials.jl:716; #invokelatest#2
    ╎    2945 ...execute_request.jl:67; execute_request(socket::ZMQ.S...
    ╎     2945 ...SoftGlobalScope.jl:65; softscope_include_string(m::...
    ╎    ╎ 2945 @Base/loading.jl:1196; include_string(mapexpr::typ...
    ╎    ╎  2945 @Base/boot.jl:373; eval
    ╎    ╎   2945 ...l/src/pyfncall.jl:86; (::PyObject)(args::Tuple{In...
    ╎    ╎    2945 .../src/pyfncall.jl:86; #_#114
    ╎    ╎     2146 .../conversions.jl:835; convert(#unused#::Type{Py...
    ╎    ╎    ╎ 2146 .../src/pyarray.jl:267; convert(#unused#::Type{A...
    ╎    ╎    ╎  2146 .../src/pyarray.jl:252; pyocopy(a::PyArray{Float...
    ╎    ╎    ╎   88   ...src/pyarray.jl:148; copy
    ╎    ╎    ╎    88   @Base/boot.jl:474; Array
    ╎    ╎    ╎

In [134]:
# nlcpy
@time x3 = vp.random.random_sample((2^17,2^10));

  0.001867 seconds (10 allocations: 464 bytes)


In [172]:
@. typeof((x0,x1,x2,x3))

(Matrix{Float64}, Matrix{Float64}, PyArray{Float64, 2}, PyObject)

In [136]:
@. size((x0,x1,x2,x3))

((131072, 1024), (131072, 1024), (131072, 1024), (131072,))

In [170]:
@. typeof((x2.o,x3))

(PyObject, PyObject)

In [168]:
x2.o.shape, x3.shape

((131072, 1024), (131072, 1024))

`PyObject` (`numpy.ndarray`) へのアクセス:

In [139]:
typeof(x2.o)

PyObject

In [16]:
x2.o.shape, py"$x2.shape"

((131072, 1024), (131072, 1024))

In [18]:
string(x2.o.__array_interface__["data"][1], base=16)

"7f91e69fa010"

In [138]:
x2.data, x2.info.pybuf.buf.buf

(Ptr{Float64} @0x00007f8f329bc010, Ptr{Nothing} @0x00007f8f329bc010)

## Move array

Julia `Array` から PyObject `numpy.ndarray` への変換はzero-copy

In [140]:
@time y2 = @pycall np.asarray(x0)::PyArray;

  0.000058 seconds (24 allocations: 1.125 KiB)


In [141]:
pointer(x0), y2.data

(Ptr{Float64} @0x00007f8fb29be040, Ptr{Float64} @0x00007f8fb29be040)

In [145]:
string(y2.o.__array_interface__["data"][1], base=16)

"7f8fb29be040"

In [146]:
strides(x0), py"$y2.strides"

((1, 131072), (8, 1048576))

In [147]:
py"$x2.shape", py"$y2.shape"

((131072, 1024), (131072, 1024))

In [148]:
py"$x2.strides", py"$y2.strides"

((8192, 8), (8, 1048576))

In [155]:
y2.data, y2.o.shape

(Ptr{Float64} @0x00007f8fb29be040, (131072, 1024))

In [156]:
@time y0 = unsafe_wrap(Array{Float64}, y2.data, *(y2.o.shape...));

  0.000042 seconds (21 allocations: 528 bytes)


In [157]:
@. size((x0,y0))

((131072, 1024), (134217728,))

In [158]:
*(size(x0)...)

134217728

In [159]:
pointer(x0), pointer(y0)

(Ptr{Float64} @0x00007f8fb29be040, Ptr{Float64} @0x00007f8fb29be040)

In [161]:
z0 = reshape(y2, size(x0));

In [162]:
@. pointer((x0,y0,z0))

(Ptr{Float64} @0x00007f8fb29be040, Ptr{Float64} @0x00007f8fb29be040, Ptr{Float64} @0x00007f8fb29be040)

In [163]:
@. size((x0,y0,z0))

((131072, 1024), (134217728,), (131072, 1024))

* `PyCall.NoCopyArray`
  - [Docstrings · PyCall.jl](https://docs.juliahub.com/PyCall/GkzkC/1.92.2/autodocs/#PyCall.NoCopyArray-Tuple{PyObject})

In [165]:
@time n0 = PyCall.NoCopyArray(x2.o);

  0.000044 seconds (30 allocations: 1.062 KiB)


In [166]:
size(n0), strides(n0)

((131072, 1024), (1024, 1))

In [167]:
x2.data, pointer(n0)

(Ptr{Float64} @0x00007f8f329bc010, Ptr{Float64} @0x00007f8f329bc010)